In [7]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()                    #input -? OUtput?    RF
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)    28*28*1   28*28*16   3
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)   28*28*16  28*28*16   5
        self.pool1 = nn.MaxPool2d(2, 2)                28*28*16  14*14*16   6
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1)   14*14*16  14*14*16   10
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1)   14*14*16  14*14*16   14
        self.pool2 = nn.MaxPool2d(2, 2)                14*14*16  7*7*16     15 
        self.conv5 = nn.Conv2d(16, 16, 3)              7*7*16    5*5*16     23 
        self.conv6 = nn.Conv2d(16, 32, 3)              5*5*16    3*3*32     32 
        self.conv7 = nn.Conv2d(32, 10, 3)              3*3*32    1*1*32     40
        self.bn16 = nn.BatchNorm2d(16)
#         self.bn8 = nn.BatchNorm2d(8)
        self.bn32 = nn.BatchNorm2d(32)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
#         x = self.bn16(x)
        x = self.conv2(x)
        x = F.relu(x)
#         x = self.bn16(x)
        x = self.pool1(x)
#       -------------------------------------------------------------------------
        x = self.conv3(x)
        x = F.relu(x)
#         x = self.bn16(x)
        x = self.conv4(x)
#         x = self.bn16(x)
        x = F.relu(x)
        x = self.pool2(x)
#       -------------------------------------------------------------------------
        x = self.conv5(x)
        x = F.relu(x)
#         x = self.bn16(x)
        x = self.conv6(x)
        x = F.relu(x)
#       -------------------------------------------------------------------------
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x,dim=1)

In [9]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Defaulting to user installation because normal site-packages is not writeable
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
            Conv2d-2           [-1, 16, 28, 28]           2,320
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           2,320
            Conv2d-5           [-1, 16, 14, 14]           2,320
         MaxPool2d-6             [-1, 16, 7, 7]               0
            Conv2d-7             [-1, 16, 5, 5]           2,320
            Conv2d-8             [-1, 32, 3, 3]           4,640
            Conv2d-9             [-1, 10, 1, 1]           2,890
Total params: 16,970
Trainable params: 16,970
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.27
Params size (MB): 0.06

In [10]:


torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [11]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [12]:

model = Net().to(device)

# optimizer = optim.SGD(model.parameters(), lr=0.015, momentum=0.9)
# # scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[7,11],gamma=0.5)
# scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[7,15],gamma=0.5)
# scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[7,11],gamma=0.5)


# Used MultistepLR to increase LR value starting from 0.015 into multiples of 0.5 
optimizer = optim.SGD(model.parameters(), lr=0.015, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[7,15],gamma=0.5)

for epoch in range(1, 21):
    print('\nEpoch: ',epoch) #print epoch value
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()
    for param_groups in optimizer.param_groups:
         print("Learning rate =",param_groups['lr']," for epoch: ",epoch+1) # print LR for different epochs

  0%|          | 0/1875 [00:00<?, ?it/s]


Epoch:  1


  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0730, Accuracy: 9781/10000 (98%)

Learning rate = 0.015  for epoch:  2

Epoch:  2


  0%|          | 0/1875 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0407, Accuracy: 9926/10000 (99%)

Learning rate = 0.00375  for epoch:  19

Epoch:  19


  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0422, Accuracy: 9929/10000 (99%)

Learning rate = 0.00375  for epoch:  20

Epoch:  20


loss=1.1920928955078125e-07 batch_id=1874: 100%|██████████| 1875/1875 [00:10<00:00, 179.85it/s]



Test set: Average loss: 0.0428, Accuracy: 9925/10000 (99%)

Learning rate = 0.00375  for epoch:  21
